In [1]:
################################################################


################################################################
# https://github.com/LiuAoyu1998/STIDGCN

# .log文件有模型结构
################################################################
# conda create -n STGNN_STIDGCN
# conda activate STGNN_STIDGCN

# conda install python=3.8

# https://pytorch.org/get-started/previous-versions/
# conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia


################################################################
# pip install pandas
# pip install scipy


################################################################
# conda install ipykernel
# conda install platformdirs
# pip3 install ipywidgets
# pip3 install --upgrade jupyter_core jupyter_client

# python -m ipykernel install --user --name STGNN_STIDGCN


################################################################
# train.py    cuda:0


################################################################

In [2]:
import torch
import torch.optim as optim

import argparse
import numpy as np
import os
import pandas as pd  # 数据分析
import random
import time

import util
from util import *
from model import STIDGCN
from ranger21 import Ranger

In [3]:
parser = argparse.ArgumentParser()

parser.add_argument("--device",     type=str, default="cuda:0", help="")

# (10699/3567/3567, 12, 170, 3)
parser.add_argument("--dataset",    type=str, default="PEMS08", help="data path")
parser.add_argument("--batch_size", type=int, default=64,       help="batch size")
# Nodes
parser.add_argument("--input_dim",  type=int, default=3,        help="number of input_dim")
# windows

# model parametres
parser.add_argument("--dropout",       type=float, default=0.1,    help="dropout rate")
parser.add_argument("--learning_rate", type=float, default=0.001,  help="learning rate")
parser.add_argument("--weight_decay",  type=float, default=0.0001, help="weight decay rate")

# training parametres
parser.add_argument("--epochs",      type=int, default=500, help="")
parser.add_argument("--print_every", type=int, default=50,  help="")
parser.add_argument("--save",        type=str, default="./logs/" + str(time.strftime("%Y-%m-%d-%H:%M:%S")) + "-", help="save path")
parser.add_argument("--expid",       type=int, default=1,   help="experiment id")
parser.add_argument("--es_patience", type=int, default=100, help="quit if no improvement after this many iterations")

args = parser.parse_args(args=[])

In [4]:
def seed_it(seed) :

    random.seed(seed)
    os.environ["PYTHONSEED"] = str(seed)
    np.random.seed(seed)

    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True
    torch.manual_seed(seed)

In [5]:
class trainer :


    def __init__(self, device, num_nodes, input_dim, channels, day_granularity, 
                 dropout, lrate, wdecay, scaler) :

        self.model = STIDGCN(device, num_nodes, input_dim, channels, day_granularity, dropout)
        self.model.to(device)

        # self.optimizer = optim.Adam(self.model.parameters(), lr=lrate, weight_decay=wdecay)
        self.optimizer = Ranger(self.model.parameters(), lr=lrate, weight_decay=wdecay)

        self.loss   = util.MAE_torch
        self.scaler = scaler
        self.clip   = 5

        print("The number of parameters: {}".format(self.model.param_num()))
        # print(self.model)


    def train(self, input, real_val) :

        self.model.train()
        self.optimizer.zero_grad()

        output = self.model(input)
        output = output.transpose(1, 3)
        predict = self.scaler.inverse_transform(output)

        real = torch.unsqueeze(real_val, dim=1)

        loss = self.loss(predict, real, 0.0)
        loss.backward()

        if self.clip is not None :  # 5
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip)
        self.optimizer.step()

        mape  = util.MAPE_torch (predict, real, 0.0).item()
        rmse  = util.RMSE_torch (predict, real, 0.0).item()
        wmape = util.WMAPE_torch(predict, real, 0.0).item()

        return loss.item(), mape, rmse, wmape


    def eval(self, input, real_val) :

        self.model.eval()

        output = self.model(input)
        output = output.transpose(1, 3)
        predict = self.scaler.inverse_transform(output)

        real = torch.unsqueeze(real_val, dim=1)

        loss = self.loss(predict, real, 0.0)

        mape  = util.MAPE_torch (predict, real, 0.0).item()
        rmse  = util.RMSE_torch (predict, real, 0.0).item()
        wmape = util.WMAPE_torch(predict, real, 0.0).item()

        return loss.item(), mape, rmse, wmape

In [6]:
seed_it(6666)

# [N, D, T] = 170*3*12
# [N, D, T] = 170*96*288
dataset  = "PEMS08"
data_dir = "data//" + dataset
num_nodes = 170
# features = 3
# windows = 12
channels = 96
day_granularity = 288

device = torch.device(args.device)

dataloader = util.load_dataset(data_dir, args.batch_size, args.batch_size, args.batch_size)

scaler = dataloader["scaler"]

Perform shuffle on the dataset


In [7]:
# 
test_log = 999999
epochs_since_best_mae = 0
path = args.save + args.dataset + "/"
if not os.path.exists(path) :
    os.makedirs(path)

# 
loss        = 9999999
his_loss    = []
train_time  = []
val_time    = []
result      = []
test_result = []

# 
engine = trainer(device, num_nodes, args.input_dim, channels, day_granularity, 
                 args.dropout, args.learning_rate, args.weight_decay, scaler)

print("start training...", flush=True)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
The number of parameters: 4423029
start training...


In [8]:
# input: [64, 12, 170, 3]
# STIDGCN(
#   (Temb): TemporalEmbedding()
#   (start_conv): Conv2d(3, 96, kernel_size=(1, 1), stride=(1, 1))
#   (tree): IDGCN_Tree(
#     (IDGCN1): IDGCN(
#       (split): Splitting()
#       (conv1): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv2): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv3): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv4): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (dgcn): DGCN(
#         (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#         (generator): Graph_Generator(
#           (fc): Linear(in_features=2, out_features=1, bias=True)
#         )
#         (gcn): Diffusion_GCN(
#           (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#           (dropout): Dropout(p=0.1, inplace=False)
#         )
#       )
#     )
#     (IDGCN2): IDGCN(
#       (split): Splitting()
#       (conv1): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv2): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv3): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv4): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (dgcn): DGCN(
#         (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#         (generator): Graph_Generator(
#           (fc): Linear(in_features=2, out_features=1, bias=True)
#         )
#         (gcn): Diffusion_GCN(
#           (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#           (dropout): Dropout(p=0.1, inplace=False)
#         )
#       )
#     )
#     (IDGCN3): IDGCN(
#       (split): Splitting()
#       (conv1): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv2): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv3): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (conv4): Sequential(
#         (0): ReplicationPad2d((3, 3, 0, 0))
#         (1): Conv2d(192, 192, kernel_size=(1, 5), stride=(1, 1))
#         (2): LeakyReLU(negative_slope=0.01, inplace=True)
#         (3): Dropout(p=0.1, inplace=False)
#         (4): Conv2d(192, 192, kernel_size=(1, 3), stride=(1, 1))
#         (5): Tanh()
#       )
#       (dgcn): DGCN(
#         (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#         (generator): Graph_Generator(
#           (fc): Linear(in_features=2, out_features=1, bias=True)
#         )
#         (gcn): Diffusion_GCN(
#           (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#           (dropout): Dropout(p=0.1, inplace=False)
#         )
#       )
#     )
#   )
#   (glu): GLU(
#     (conv1): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#     (conv2): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#     (conv3): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
#     (dropout): Dropout(p=0.1, inplace=False)
#   )
#   (regression_layer): Conv2d(192, 12, kernel_size=(1, 12), stride=(1, 1))
# )

In [9]:
for i in range(1, args.epochs + 1) :

    train_loss  = []
    train_mape  = []
    train_rmse  = []
    train_wmape = []

    t1 = time.time()

    for iter, (x, y) in enumerate(dataloader["train_loader"].get_iterator()) :  # 返回批数据

        trainx = torch.Tensor(x).to(device)
        # print(trainx.shape)  # torch.Size([64, 12, 170, 3])
        trainx = trainx.transpose(1, 3)
        trainy = torch.Tensor(y).to(device)
        trainy = trainy.transpose(1, 3)

        metrics = engine.train(trainx, trainy[:, 0, :, :])
        train_loss.append(metrics[0])
        train_mape.append(metrics[1])
        train_rmse.append(metrics[2])
        train_wmape.append(metrics[3])

        if iter % args.print_every == 0 :
            log = "Iter: {:03d}, \
                   Train Loss: {:.4f}, \
                   Train RMSE: {:.4f}, \
                   Train MAPE: {:.4f}, \
                   Train WMAPE: {:.4f}"
            print(
                log.format(
                    iter, 
                    train_loss[-1], 
                    train_rmse[-1], 
                    train_mape[-1], 
                    train_wmape[-1], 
                ), 
                flush=True, 
            )

    t2 = time.time()
    log = "Epoch: {:03d}, Training Time: {:.4f} secs"
    print(log.format(i, (t2 - t1)))
    train_time.append(t2 - t1)

    valid_loss  = []
    valid_mape  = []
    valid_wmape = []
    valid_rmse  = []

    # 
    s1 = time.time()

    for iter, (x, y) in enumerate(dataloader["val_loader"].get_iterator()) :

        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1, 3)
        testy = torch.Tensor(y).to(device)
        testy = testy.transpose(1, 3)

        metrics = engine.eval(testx, testy[:, 0, :, :])
        valid_loss .append(metrics[0])
        valid_mape .append(metrics[1])
        valid_rmse .append(metrics[2])
        valid_wmape.append(metrics[3])

    s2 = time.time()

    log = "Epoch: {:03d}, Inference Time: {:.4f} secs"
    print(log.format(i, (s2 - s1)))
    val_time.append(s2 - s1)

    mtrain_loss  = np.mean(train_loss)
    mtrain_mape  = np.mean(train_mape)
    mtrain_wmape = np.mean(train_wmape)
    mtrain_rmse  = np.mean(train_rmse)

    mvalid_loss  = np.mean(valid_loss)
    mvalid_mape  = np.mean(valid_mape)
    mvalid_wmape = np.mean(valid_wmape)
    mvalid_rmse  = np.mean(valid_rmse)

    his_loss.append(mvalid_loss)
    train_m = dict(
        train_loss =np.mean(train_loss), 
        train_rmse =np.mean(train_rmse), 
        train_mape =np.mean(train_mape), 
        train_wmape=np.mean(train_wmape), 

        valid_loss =np.mean(valid_loss), 
        valid_rmse =np.mean(valid_rmse), 
        valid_mape =np.mean(valid_mape), 
        valid_wmape=np.mean(valid_wmape), 
    )
    train_m = pd.Series(train_m)
    result.append(train_m)

    log = "Epoch: {:03d}, \
           Train Loss: {:.4f}, \
           Train RMSE: {:.4f}, \
           Train MAPE: {:.4f}, \
           Train WMAPE: {:.4f}, "
    print(
        log.format(i, mtrain_loss, mtrain_rmse, mtrain_mape, mtrain_wmape), 
        flush=True, 
    )
    log = "Epoch: {:03d}, \
           Valid Loss: {:.4f}, \
           Valid RMSE: {:.4f}, \
           Valid MAPE: {:.4f}, \
           Valid WMAPE: {:.4f}"
    print(
        log.format(i, mvalid_loss, mvalid_rmse, mvalid_mape, mvalid_wmape), 
        flush=True, 
    )

    if mvalid_loss < loss :
        print("### Update tasks appear ###")

        if i < 100 :
            # It is not necessary to print the results of the test set 
            # when epoch is less than 100, because the model has not yet converged.
            loss = mvalid_loss
            torch.save(engine.model.state_dict(), path + "best_model.pth")
            bestid = i
            epochs_since_best_mae = 0
            print("Updating! Valid Loss:", mvalid_loss, end=", ")
            print("epoch: ", i)

        elif i > 100 :
            outputs = []
            realy = torch.Tensor(dataloader["y_test"]).to(device)
            realy = realy.transpose(1, 3)[:, 0, :, :]

            for iter, (x, y) in enumerate(dataloader["test_loader"].get_iterator()) :
                testx = torch.Tensor(x).to(device)
                testx = testx.transpose(1, 3)
                with torch.no_grad() :
                    preds = engine.model(testx).transpose(1, 3)
                outputs.append(preds.squeeze())

            yhat = torch.cat(outputs, dim=0)
            yhat = yhat[: realy.size(0), ...]

            amae   = []
            amape  = []
            awmape = []
            armse  = []
            test_m = []

            for j in range(12) :

                pred = scaler.inverse_transform(yhat[:, :, j])
                real = realy[:, :, j]
                metrics = util.metric(pred, real)
                log = "Evaluate best model on test data for horizon {:d}, \
                    Test MAE  : {:.4f}, \
                    Test RMSE : {:.4f}, \
                    Test MAPE : {:.4f}, \
                    Test WMAPE: {:.4f}"
                print(
                    log.format(
                        j + 1, metrics[0], metrics[2], metrics[1], metrics[3]
                    )
                )

                test_m = dict(
                    test_loss =np.mean(metrics[0]), 
                    test_rmse =np.mean(metrics[2]), 
                    test_mape =np.mean(metrics[1]), 
                    test_wmape=np.mean(metrics[3]), 
                )
                test_m = pd.Series(test_m)

                amae  .append(metrics[0])
                amape .append(metrics[1])
                armse .append(metrics[2])
                awmape.append(metrics[3])

            log = "On average over 12 horizons, \
                   Test MAE  : {:.4f}, \
                   Test RMSE : {:.4f}, \
                   Test MAPE : {:.4f}, \
                   Test WMAPE: {:.4f}"
            print(
                log.format(
                    np.mean(amae), np.mean(armse), np.mean(amape), np.mean(awmape)
                )
            )

            if np.mean(amae) < test_log :
                test_log = np.mean(amae)
                loss = mvalid_loss
                torch.save(engine.model.state_dict(), path + "best_model.pth")
                epochs_since_best_mae = 0
                print("Test low! Updating! Test Loss :", np.mean(amae), end=", ")
                print("Test low! Updating! Valid Loss:", mvalid_loss  , end=", ")
                bestid = i
                print("epoch: ", i)
            else :
                epochs_since_best_mae += 1
                print("No update")

    else :
        epochs_since_best_mae += 1
        print("No update")

    train_csv = pd.DataFrame(result)
    train_csv.round(8).to_csv(f"{path}/train.csv")
    if epochs_since_best_mae >= args.es_patience and i >= 300 :
        break

torch.Size([64, 96, 170, 12])
Iter: 000,                    Train Loss: 118.7981,                    Train RMSE: 145.8948,                    Train MAPE: 2.0573,                    Train WMAPE: 0.4951


/home/zhfc/github files/4.1-Graph-Neural-Networks/4-Spatio-Temporal GNN/2-model/2205-Spatial-Temporal Dynamic Graph Convolutional Network With Interactive Learning for Traffic Forecasting/STGNN_STIDGCN/ranger21.py:139: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg_sq.mul_(beta2).addcmul_(1-beta2, grad, grad)


torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size([64, 96, 170, 12])
torch.Size

KeyboardInterrupt: 

In [ ]:


# print("Average Training Time : {:.4f} secs/epoch".format(np.mean(train_time)))
# print("Average Inference Time: {:.4f} secs".format(np.mean(val_time)))

# print("Training ends")
# print("The epoch of the best result：", bestid)
# print("The valid loss of the best model", str(round(his_loss[bestid - 1], 4)))

# engine.model.load_state_dict(torch.load(path + "best_model.pth"))
# outputs = []
# realy = torch.Tensor(dataloader["y_test"]).to(device)
# realy = realy.transpose(1, 3)[:, 0, :, :]

# for iter, (x, y) in enumerate(dataloader["test_loader"].get_iterator()) :
#     testx = torch.Tensor(x).to(device)
#     testx = testx.transpose(1, 3)
#     with torch.no_grad() :
#         preds = engine.model(testx).transpose(1, 3)
#     outputs.append(preds.squeeze())

# yhat = torch.cat(outputs, dim=0)
# yhat = yhat[: realy.size(0), ...]

# amae   = []
# amape  = []
# armse  = []
# awmape = []

# test_m = []

# for i in range(12) :
#     pred = scaler.inverse_transform(yhat[:, :, i])
#     real = realy[:, :, i]
#     metrics = util.metric(pred, real)
#     log = "Evaluate best model on test data for horizon {:d}, \
#            Test MAE: {:.4f}, \
#            Test RMSE: {:.4f}, \
#            Test MAPE: {:.4f}, \
#            Test WMAPE: {:.4f}"
#     print(log.format(i + 1, metrics[0], metrics[2], metrics[1], metrics[3]))

#     test_m = dict(
#         test_loss =np.mean(metrics[0]), 
#         test_rmse =np.mean(metrics[2]), 
#         test_mape =np.mean(metrics[1]), 
#         test_wmape=np.mean(metrics[3]), 
#     )
#     test_m = pd.Series(test_m)
#     test_result.append(test_m)

#     amae  .append(metrics[0])
#     amape .append(metrics[1])
#     armse .append(metrics[2])
#     awmape.append(metrics[3])

# log = "On average over 12 horizons, \
#        Test MAE: {:.4f}, \
#        Test RMSE: {:.4f}, \
#        Test MAPE: {:.4f}, \
#        Test WMAPE: {:.4f}"
# print(log.format(np.mean(amae), np.mean(armse), np.mean(amape), np.mean(awmape)))

# test_m = dict(
#     test_loss =np.mean(amae), 
#     test_rmse =np.mean(armse), 
#     test_mape =np.mean(amape), 
#     test_wmape=np.mean(awmape), 
# )
# test_m = pd.Series(test_m)
# test_result.append(test_m)

# test_csv = pd.DataFrame(test_result)
# test_csv.round(8).to_csv(f"{path}/test.csv")